In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import f 
import random
pd.set_option('display.max_rows', 10)

查表 http://eschool.kuas.edu.tw/tsungo/Publish/Appendix.pdf

# 匯入資料&抽樣 (僅第一次)

    半導體業(SEMI)與營建業(BUILD)各抽30家，抽出放回。

# 範例1:單母體平均值檢定

    虛無假設H0 : μ = 4
    對立假設H1 : μ ≠ 4
    
    設定顯著水準 α = 0.05，檢驗半導體類股的殖利率平均值是否 = 4%?

In [4]:
#讀入半導體類股殖利率資料
SEMI = pd.read_csv("SEMI_30.csv")
SEMI.head()

,Security Code,Dividend yield (%)
0,2481,1.98
1,3583,2.99
2,2481,1.98
3,6533,0.07
4,2481,1.98


In [5]:
#單母體平均值檢定
#檢定半導體類股的殖利率是否平均數為4
stats.ttest_1samp(SEMI['Dividend yield (%)'], 4,alternative="two-sided")

Ttest_1sampResult(statistic=-5.188360225453648, pvalue=1.5026908746166407e-05)

    (1) statistic = -5.188360225453648，拒絕域為 /t/> t臨界值(自由度29) = 2.045 
        abs(-5.188360225453648) = 5.188360225453648 > 2.045，
        檢定統計量的值落於拒絕域中
    (2) pvalue=1.5026908746166407e-05 < 0.05，拒絕虛無假設
        
    結論 : 在0.05的顯著水準下，半導體類股殖利率平均值不為4%。

# 範例2:單母體變異數檢定 

    虛無假設H0 : σ^2 = 1
    對立假設H1 : σ^2 ≠ 1
    
    設定顯著水準α = 0.05，檢驗建材營造類股的殖利率變異數是否 = 15?

In [21]:
#讀入建材營造類股的殖利率資料
BUILD = pd.read_csv("BUILD_30.csv")
BUILD.head()

,Security Code,Dividend yield (%)
0,6177,9.62
1,2530,0.71
2,5525,5.96
3,3052,3.67
4,1808,5.22


In [22]:
#單母體變異數檢定
#檢定建材營造類股的殖利率變異數是否 = 15

#計算樣本變異數
var = np.var(BUILD[['Dividend yield (%)']], ddof = 1) 

#計算統計量
chi_square = ((29) * var) / 15
chi_square

Dividend yield (%)    14.484418
dtype: float64

In [23]:
#計算p-vaule，自由度為29
from scipy.stats import chi2

p_value = chi2.cdf(x = chi_square, df = 29)
p_value

array([0.01136435])

    (1) 拒絕域為 卡方統計量 > 45.7223 或 < 16.0471
        statistic = 14.484418 < 16.0471  
        檢定統計量的值落於拒絕域中   
    (2) pvalue = 0.01136435 < 0.05，拒絕虛無假設

    結論 : 在0.05的顯著水準下，建材營造類股 的 殖利率 變異數不為15。

# 範例3:單母體比例檢定

    H0 : p = p0
    H1 : p ≠ p1
    
    設定顯著水準α = 0.05，調查半導體業殖利率大於 4% 的比例約 0.5 ，隨機抽取30位，
    得知殖利率大於 4% 的比例為 0.1，調查結果是否可靠?

In [24]:
p = 0.5
q = 1-p
sigma = ( (p*q) / 30 ) ** 0.5
z = (0.1-0.5) / sigma
z

-4.3817804600413295

    z= -4.3817804600413295 < 臨界值-1.96，落入拒絕域
    
    結論 :  在0.05的顯著水準下，半導體業殖利率大於 4% 的比例不為 0.5

# 範例4:雙母體的平均值檢定

    H0 : μ1 = μ2
    H1 : μ1 ≠ μ2
    
    設定顯著水準α = 0.05，檢驗半導體與營建業的殖利率 平均數是否有差異?

In [25]:
#合併半導體與營建業 殖利率資料
dividends = pd.concat([SEMI, BUILD], axis = 1)
dividends.drop('Security Code', axis = 1, inplace = True)
dividends.columns = ['SEMI', 'BUILD']
dividends

,SEMI,BUILD
0,1.98,9.62
1,2.99,0.71
2,1.98,5.96
3,0.07,3.67
4,1.98,5.22
...,...,...
25,4.56,5.37
26,2.94,3.82
27,1.91,2.78
28,0.35,3.67


In [167]:
# 個別指派變數
SEMI_D_ratio=[dividends.SEMI]
BUILD_D_ratio=[dividends.BUILD]

In [26]:
stats.ttest_ind(SEMI[['Dividend yield (%)']], BUILD[['Dividend yield (%)']], equal_var = True, alternative='two-sided')

Ttest_indResult(statistic=array([-3.46563867]), pvalue=array([0.00100212]))

    (1) statistic = -3.46563867 < -1.96，檢定統計量的值落入拒絕域中
    (2) pvalue = 0.00100212 < 0.05，在0.05顯著水準下，拒絕虛無假設
    
    結論 : 在0.05的顯著水準下，半導體與營建業的殖利率 平均數不同。

# 範例5:雙母體的變異數檢定

    H0 : μ1^2 = μ2^2
    H1 : μ1^2 ≠ μ2^2
    
    左尾檢定
    設定顯著水準α = 0.05，檢驗半導體業與營建業整體殖利率的 變異數是否相等?

In [27]:
from scipy.stats import f

n1 = 30; n2 = 30
alpha = 0.05
var_SEMI = np.var(SEMI[['Dividend yield (%)']], ddof = 1) 
var_BUILD = np.var(BUILD[['Dividend yield (%)']], ddof = 1)
fc = f.ppf(alpha, n1-1, n2-1)
fc #臨界值

0.5373999648406917

In [28]:
f0 = var_SEMI / var_BUILD
f0 #檢定統計量   
#統計量 < 臨界值，落入拒絕域

Dividend yield (%)    0.498886
dtype: float64

In [30]:
#p-value
pvalue = f.cdf(f0, n1-1, n2-1)
pvalue

array([0.03301329])

    pvalue = 0.03301329 < 0.05，落入拒絕域
    結論 : 半導體業與營建業整體殖利率 變異數不同

# 範例6:雙母體比例檢定

    H0 : p1 - p2 <= 0
    H1 : p1 - p2 >  0
    
    右尾檢定
    調查建議投資半導體還是營建業
    隨機抽出半導體 30 家，殖利率大於 4% ，有 3 家
    再隨機抽出營建業 30 家，殖利率大於 4% ，有 14 家
    檢定半導體中殖利率大於 4% 的比例是否大於營建業

In [194]:
from scipy.stats import norm

p0 = 0
n1 = len(BUILD); n2 = len(SEMI)
pbar1 = 3 /  len(SEMI) #半導體
pbar2 = 14 / len(BUILD)  #營建業
pbar = (n1 * pbar1 + n2 * pbar2 ) / ( n1 + n2 )
se = np.sqrt( pbar * ( 1 - pbar ) * ( 1 / n1 + 1 / n2 ) )
pbar0 = pbar1 - pbar2
alpha = 0.05
zc = norm.ppf(1-alpha, 0,1)
zc #臨界值

1.6448536269514722

In [195]:
#pbar c = p0 + zc * se
z0 = ( pbar0 - p0 ) / se
z0

-3.1514442002351277

In [197]:
#p-value
1-norm.cdf(pbar0,p0,se)
1-norm.cdf(z0,0,1)

0.9991876739891977

    pvalue = 0.9991876739891977 > 0.05，不落入拒絕域
    結論 : 半導體中殖利率大於 4% 的比例 不高於營建業